In [49]:
import logging.config
import unittest
from UtilityInspect import whoami, whosdaddy, listObject
import pandas as pd
print(pd.__version__)

0.13.1


Load the file

In [50]:
excel_path = r"D:\Dropbox\03 Side project\00 iC\Forecast.xlsx"
logging.debug("XLSX file at {}".format(excel_path))

xlsx = pd.ExcelFile(excel_path)
logging.debug("XLSX object {} with sheets {}".format(xlsx,xlsx.sheet_names))
#hours = xlsx.parse('Hours',header=2,skiprows=2,parse_dates=True)
hours = xlsx.parse('Hours',skiprows=2,header=2)
hours = hours.transpose()
hours.head()

NaN                                                     \
                  13319         13330                                13317   
            Dubai Villa        Kosovo       Konrad, OMV Biozid rohrplanung   
                Stunden  Tag  Stunden  Tag                         Stunden   
2014-01-01          NaN  NaN        5  NaN                             NaN   
2014-01-02          NaN  NaN      NaN  NaN                             NaN   
2014-01-03          NaN  NaN      NaN  NaN                             NaN   
2014-01-04          NaN  NaN      NaN  NaN                             NaN   
2014-01-05          NaN  NaN      NaN  NaN                             NaN   

                               
                       A       
                     FFG       
            Tag  Stunden  Tag  
2014-01-01  NaN      NaN  NaN  
2014-01-02  NaN      NaN  NaN  
2014-01-03  NaN      NaN  NaN  
2014-01-04  NaN      NaN  NaN  
2014-01-05  NaN      NaN  NaN  

[5 rows x 8 columns]

Create names

In [51]:
hours.columns.names = [None, 'Number', 'Name', 'Type']

In [52]:
assert(hours.index.dtype == 'datetime64[ns]')
hours.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 74 entries, 2014-01-01 00:00:00 to 2014-03-15 00:00:00
Data columns (total 8 columns):
(nan, 13319, Dubai Villa, Stunden)                       1 non-null float64
(nan, 13319, Dubai Villa, Tag)                           0 non-null float64
(nan, 13330, Kosovo, Stunden)                            11 non-null float64
(nan, 13330, Kosovo, Tag)                                10 non-null float64
(nan, 13317, Konrad, OMV Biozid rohrplanung, Stunden)    7 non-null float64
(nan, 13317, Konrad, OMV Biozid rohrplanung, Tag)        0 non-null float64
(nan, A, FFG, Stunden)                                   7 non-null float64
(nan, A, FFG, Tag)                                       0 non-null float64
dtypes: float64(8)

In [53]:
# Levels
hours.columns.levels[:]

FrozenList([[], [13317, 13319, 13330, u'A'], [u'Dubai Villa', u'FFG', u'Konrad, OMV Biozid rohrplanung', u'Kosovo'], [u'Stunden', u'Tag']])

In [54]:
# Names
hours.columns.names[:]

FrozenList([None, u'Number', u'Name', u'Type'])

In [55]:
# Labels
hours.columns.labels[:]

FrozenList([[-1, -1, -1, -1, -1, -1, -1, -1], [1, 1, 2, 2, 0, 0, 3, 3], [0, 0, 3, 3, 2, 2, 1, 1], [0, 1, 0, 1, 0, 1, 0, 1]])

Remove the first column row

In [56]:
new_levels = hours.columns.levels[1:]
#new_labels = [l.tolist()  for l in hours.columns.labels[:]] 
new_labels = hours.columns.labels[1:]
new_names = hours.columns.names[1:]
print(new_levels)
print(new_labels)

[[13317, 13319, 13330, u'A'], [u'Dubai Villa', u'FFG', u'Konrad, OMV Biozid rohrplanung', u'Kosovo'], [u'Stunden', u'Tag']]
[[1, 1, 2, 2, 0, 0, 3, 3], [0, 0, 3, 3, 2, 2, 1, 1], [0, 1, 0, 1, 0, 1, 0, 1]]


In [57]:
new_index = pd.MultiIndex(levels=new_levels,
           labels=new_labels,
           names=new_names)
hours.columns = new_index

In [58]:
hours.head()

Number            13319         13330                                13317  \
Name        Dubai Villa        Kosovo       Konrad, OMV Biozid rohrplanung   
Type            Stunden  Tag  Stunden  Tag                         Stunden   
2014-01-01          NaN  NaN        5  NaN                             NaN   
2014-01-02          NaN  NaN      NaN  NaN                             NaN   
2014-01-03          NaN  NaN      NaN  NaN                             NaN   
2014-01-04          NaN  NaN      NaN  NaN                             NaN   
2014-01-05          NaN  NaN      NaN  NaN                             NaN   

Number                 A       
Name                 FFG       
Type        Tag  Stunden  Tag  
2014-01-01  NaN      NaN  NaN  
2014-01-02  NaN      NaN  NaN  
2014-01-03  NaN      NaN  NaN  
2014-01-04  NaN      NaN  NaN  
2014-01-05  NaN      NaN  NaN  

[5 rows x 8 columns]

Group by months

In [59]:
months = hours.groupby(lambda x : x.month).sum()
#months.index
months.index.name = 'Month'
months

Number        13319         13330                                13317       \
Name    Dubai Villa        Kosovo       Konrad, OMV Biozid rohrplanung        
Type        Stunden  Tag  Stunden  Tag                         Stunden  Tag   
Month                                                                         
1               NaN  NaN       29    5                              11  NaN   
2                 5  NaN       12    5                              19  NaN   
3               NaN  NaN      NaN  NaN                             NaN  NaN   

Number        A       
Name        FFG       
Type    Stunden  Tag  
Month                 
1            27  NaN  
2           NaN  NaN  
3           NaN  NaN  

[3 rows x 8 columns]

In [60]:
days_worked = months.transpose().query('Type == "Tag"')
days_worked

,,Month,1,2,3
Number,Name,Type,,,
13319,Dubai Villa,Tag,NaN,NaN,NaN
13330,Kosovo,Tag,5,5,NaN
13317,"Konrad, OMV Biozid rohrplanung",Tag,NaN,NaN,NaN
A,FFG,Tag,NaN,NaN,NaN


In [61]:
hours_worked = months.transpose().query('Type == "Stunden"').transpose()
hours_worked

Number,13319,13330,13317,A
Name,Dubai Villa,Kosovo,"Konrad, OMV Biozid rohrplanung",FFG
Type,Stunden,Stunden,Stunden,Stunden
Month,,,,
1,NaN,29,11,27
2,5,12,19,NaN
3,NaN,NaN,NaN,NaN
